In [1]:
from pyspark.sql import SparkSession

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
spark = SparkSession \
    .builder \
    .appName('spark-nb') \
    .master('spark://spark-master:7077') \
    .config('spark.ui.port', '4051') \
    .getOrCreate()

2023-10-30 18:25:22,903 INFO spark.SparkContext: Running Spark version 3.4.1
2023-10-30 18:25:23,255 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-10-30 18:25:23,408 INFO resource.ResourceUtils: ==============================================================
2023-10-30 18:25:23,409 INFO resource.ResourceUtils: No custom resources configured for spark.driver.
2023-10-30 18:25:23,409 INFO resource.ResourceUtils: ==============================================================
2023-10-30 18:25:23,409 INFO spark.SparkContext: Submitted application: spark-nb
2023-10-30 18:25:23,479 INFO resource.ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
2023-10-30 18:25:23,

In [3]:
!mkdir -p /home/data/nyc_taxi_data/green_tripdata && \
    rm -f /home/data/nyc_taxi_data/green_tripdata/* && \
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -P /home/data/nyc_taxi_data/green_tripdata/

--2023-10-30 18:25:28--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.244.115.107, 18.244.115.167, 18.244.115.202, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.244.115.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: '/home/data/nyc_taxi_data/green_tripdata/green_tripdata_2023-01.parquet'

100%[======================================>] 1,427,002   1.22MB/s   in 1.1s   

2023-10-30 18:25:31 (1.22 MB/s) - '/home/data/nyc_taxi_data/green_tripdata/green_tripdata_2023-01.parquet' saved [1427002/1427002]



In [4]:
df = spark.read.format('parquet').load('/home/data/nyc_taxi_data/green_tripdata/')

2023-10-30 18:25:31,763 INFO internal.SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
2023-10-30 18:25:31,768 INFO internal.SharedState: Warehouse path is 'file:/home/spark-warehouse'.
2023-10-30 18:25:31,800 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@6558dea2{/SQL,null,AVAILABLE,@Spark}
2023-10-30 18:25:31,801 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@c0e13fd{/SQL/json,null,AVAILABLE,@Spark}
2023-10-30 18:25:31,802 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@3edbddf8{/SQL/execution,null,AVAILABLE,@Spark}
2023-10-30 18:25:31,803 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@56caab05{/SQL/execution/json,null,AVAILABLE,@Spark}
2023-10-30 18:25:31,809 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@4de71bcd{/static/sql,null,AVAILABLE,@Spark}
2023-10-30 18:25:34,480 INFO datasources.InMemoryFileIndex: It took 59 ms to list leaf 

In [5]:
df.count()

2023-10-30 18:25:52,827 INFO datasources.FileSourceStrategy: Pushed Filters: 
2023-10-30 18:25:52,828 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2023-10-30 18:25:54,052 INFO codegen.CodeGenerator: Code generated in 663.514635 ms
2023-10-30 18:25:54,132 INFO memory.MemoryStore: Block broadcast_1 stored as values in memory (estimated size 487.1 KiB, free 365.8 MiB)
2023-10-30 18:25:54,147 INFO memory.MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 53.7 KiB, free 365.8 MiB)
2023-10-30 18:25:54,147 INFO storage.BlockManagerInfo: Added broadcast_1_piece0 in memory on 9dc5d8af21bf:44443 (size: 53.7 KiB, free: 366.2 MiB)
2023-10-30 18:25:54,149 INFO spark.SparkContext: Created broadcast 1 from count at NativeMethodAccessorImpl.java:0
2023-10-30 18:25:54,176 INFO execution.FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
2023-10-30 18:25:54,348 INFO scheduler.DAGSchedule

68211

In [6]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [7]:
from pyspark.sql.functions import date_format

# fix error in pandas 2.1.1
# Casting to unit-less dtype 'datetime64' is not supported. Pass e.g. 'datetime64[ns]' instead.
df = df.withColumn("lpep_pickup_datetime", date_format("lpep_pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
df = df.withColumn("lpep_dropoff_datetime", date_format("lpep_dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))

In [8]:
df.limit(10).toPandas()

2023-10-30 18:25:58,179 INFO datasources.FileSourceStrategy: Pushed Filters: 
2023-10-30 18:25:58,179 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2023-10-30 18:25:58,354 INFO codegen.CodeGenerator: Code generated in 105.810509 ms
2023-10-30 18:25:58,364 INFO memory.MemoryStore: Block broadcast_4 stored as values in memory (estimated size 492.7 KiB, free 365.3 MiB)
2023-10-30 18:25:58,374 INFO memory.MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 54.6 KiB, free 365.2 MiB)
2023-10-30 18:25:58,376 INFO storage.BlockManagerInfo: Added broadcast_4_piece0 in memory on 9dc5d8af21bf:44443 (size: 54.6 KiB, free: 366.2 MiB)
2023-10-30 18:25:58,377 INFO spark.SparkContext: Created broadcast 4 from toPandas at /tmp/ipykernel_20/1309501217.py:1
2023-10-30 18:25:58,378 INFO execution.FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
2023-10-30 18:25:58,402 INFO spark.SparkConte

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.00,0.5,4.03,0.0,NaN,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,1.00,0.5,2.64,0.0,NaN,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,1.00,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,0.50,1.5,1.70,0.0,NaN,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,0.50,1.5,0.00,0.0,NaN,1.0,8.00,1.0,1.0,0.00
5,2,2023-01-01 00:53:31,2023-01-01 01:11:04,N,1.0,41,262,1.0,2.78,17.7,1.00,0.5,0.00,0.0,NaN,1.0,22.95,2.0,1.0,2.75
6,1,2023-01-01 00:09:14,2023-01-01 00:26:39,N,1.0,181,45,2.0,3.80,19.1,3.75,1.5,4.85,0.0,NaN,1.0,29.20,1.0,1.0,2.75
7,2,2023-01-01 00:11:58,2023-01-01 00:24:55,N,1.0,24,75,1.0,1.88,14.2,1.00,0.5,0.00,0.0,NaN,1.0,16.70,2.0,1.0,0.00
8,2,2023-01-01 00:41:29,2023-01-01 00:46:26,N,1.0,41,166,2.0,1.11,7.2,1.00,0.5,1.00,0.0,NaN,1.0,10.70,1.0,1.0,0.00
9,2,2023-01-01 00:50:32,2023-01-01 01:13:42,N,1.0,24,140,1.0,4.22,24.7,1.00,0.5,3.00,0.0,NaN,1.0,32.95,1.0,1.0,2.75


In [9]:
df.limit(1000).write.format('parquet').mode('overwrite').save('s3a://spark-warehouse/raw_data/save_from_notebook')

2023-10-30 18:26:00,411 INFO storage.BlockManagerInfo: Removed broadcast_5_piece0 on 9dc5d8af21bf:44443 in memory (size: 8.6 KiB, free: 366.2 MiB)
2023-10-30 18:26:00,417 INFO storage.BlockManagerInfo: Removed broadcast_5_piece0 on 172.20.0.4:39963 in memory (size: 8.6 KiB, free: 366.2 MiB)
2023-10-30 18:26:00,430 INFO storage.BlockManagerInfo: Removed broadcast_1_piece0 on 9dc5d8af21bf:44443 in memory (size: 53.7 KiB, free: 366.2 MiB)
2023-10-30 18:26:00,435 INFO storage.BlockManagerInfo: Removed broadcast_1_piece0 on 172.20.0.4:39963 in memory (size: 53.7 KiB, free: 366.2 MiB)
2023-10-30 18:26:00,452 INFO storage.BlockManagerInfo: Removed broadcast_3_piece0 on 9dc5d8af21bf:44443 in memory (size: 5.9 KiB, free: 366.2 MiB)
2023-10-30 18:26:00,461 INFO storage.BlockManagerInfo: Removed broadcast_3_piece0 on 172.20.0.4:39963 in memory (size: 5.9 KiB, free: 366.2 MiB)
2023-10-30 18:26:00,472 INFO storage.BlockManagerInfo: Removed broadcast_2_piece0 on 9dc5d8af21bf:44443 in memory (size: 7

In [10]:
spark.stop()

2023-10-30 18:26:16,286 INFO spark.SparkContext: SparkContext is stopping with exitCode 0.
2023-10-30 18:26:16,330 INFO server.AbstractConnector: Stopped Spark@2c536ca7{HTTP/1.1, (http/1.1)}{0.0.0.0:4051}
2023-10-30 18:26:16,337 INFO ui.SparkUI: Stopped Spark web UI at http://9dc5d8af21bf:4051
2023-10-30 18:26:16,346 INFO cluster.StandaloneSchedulerBackend: Shutting down all executors
2023-10-30 18:26:16,347 INFO cluster.StandaloneSchedulerBackend$StandaloneDriverEndpoint: Asking each executor to shut down
2023-10-30 18:26:16,415 INFO spark.MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
2023-10-30 18:26:16,443 INFO memory.MemoryStore: MemoryStore cleared
2023-10-30 18:26:16,446 INFO storage.BlockManager: BlockManager stopped
2023-10-30 18:26:16,457 INFO storage.BlockManagerMaster: BlockManagerMaster stopped
2023-10-30 18:26:16,472 INFO scheduler.OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
2023-10-30 18:26:16,498 INF